In [14]:
import requests
import pandas as pd
from datetime import datetime

# API key and base URL
API_KEY = 'fe0822a03c94eb4e6df4b4685a08824546fa1ceee2ba8f67cc3a8a929e12552f'
BASE_URL_BTC_PRICE = "https://min-api.cryptocompare.com/data/v2/histoday"
BASE_URL_EXCHANGE = "https://min-api.cryptocompare.com/data/exchange/histoday"
BASE_URL_BTC_EXCHANGE = "https://min-api.cryptocompare.com/data/symbol/histoday"


In [33]:
def get_data(url, params):
    response = requests.get(url, params=params)
    data = response.json()
    print(data)
    if 'Data' in data and 'Data' in data['Data']:
        return pd.DataFrame(data['Data']['Data'])
    elif 'Data' in data:
        return pd.DataFrame(data['Data'])
    else:
        print("Error fetching data")
        return None

def get_bitcoin_data(to_ts=None):
    params = {
        'fsym': 'BTC',
        'tsym': 'USD',
        'limit': 2000,
        'api_key': API_KEY
    }
    if to_ts:
        params['toTs'] = to_ts

    return get_data(BASE_URL_BTC_PRICE, params)

def get_exchange_volume(to_ts=None):
    params = {
        'tsym': 'USD',
        'limit': 2000,
        'api_key': API_KEY
    }
    if to_ts:
        params['toTs'] = to_ts

    return get_data(BASE_URL_EXCHANGE, params)

def get_bitcoin_exchange_volume(to_ts=None):
    params = {
        'fsym': 'BTC',
        'tsym': 'USD',
        'limit': 2000,
        'api_key': API_KEY
    }
    if to_ts:
        params['toTs'] = to_ts
    return get_data(BASE_URL_BTC_EXCHANGE, params)


def save_data():
    end_date = datetime(2016, 1, 1)
    to_ts = int(datetime.now().timestamp())
    
    all_price_data = pd.DataFrame()
    all_market_volume_data = pd.DataFrame()
    all_volume_data = pd.DataFrame()
    
    while True:
        price_data = get_bitcoin_data(to_ts)
        market_volume_data = get_exchange_volume(to_ts)
        volume_data = get_bitcoin_exchange_volume(to_ts)

        if price_data is not None and market_volume_data is not None:
            price_data['time'] = pd.to_datetime(price_data['time'], unit='s')
            market_volume_data['time'] = pd.to_datetime(market_volume_data['time'], unit='s')
            volume_data['time'] = pd.to_datetime(volume_data['time'], unit='s')

            all_price_data = pd.concat([price_data, all_price_data])
            all_market_volume_data = pd.concat([market_volume_data, all_market_volume_data])
            all_volume_data = pd.concat([volume_data, all_volume_data])

            to_ts = price_data['time'].min().timestamp()
            
            if price_data['time'].min() <= end_date:
                break
        else:
            break

    all_temp = all_price_data.merge(all_volume_data, on='time', suffixes=('_price', '_volume'))
    all_data = all_temp.merge(all_market_volume_data, on='time', suffixes=('_btc', '_market'))
    all_data.to_csv('bitcoin_full_data.csv')
    print("Data saved successfully")

In [26]:
to_ts = int(datetime.now().timestamp())
price_data = get_bitcoin_data(to_ts)


{'Response': 'Success', 'Message': '', 'HasWarning': False, 'Type': 100, 'RateLimit': {}, 'Data': {'Aggregated': False, 'TimeFrom': 1543276800, 'TimeTo': 1716076800, 'Data': [{'time': 1543276800, 'high': 3878.9, 'low': 3613.11, 'open': 3784.77, 'volumefrom': 147999.31, 'volumeto': 555212127.53, 'close': 3822.99, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1543363200, 'high': 4398.57, 'low': 3822.94, 'open': 3822.99, 'volumefrom': 175435.76, 'volumeto': 726808596.57, 'close': 4263.55, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1543449600, 'high': 4444.39, 'low': 4130.68, 'open': 4263.52, 'volumefrom': 112103.36, 'volumeto': 479575430.64, 'close': 4286.69, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1543536000, 'high': 4331.07, 'low': 3916.3, 'open': 4286.85, 'volumefrom': 121993.57, 'volumeto': 497762735.43, 'close': 4009.67, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1543622400, 'high': 4309.65, 'low': 3942.62, '

In [27]:
market_volume_data = get_exchange_volume(to_ts)

{'Type': 100, 'Message': 'Got the data', 'Data': [{'time': 1543276800, 'volume': 6656715887.91}, {'time': 1543363200, 'volume': 8409566802.42}, {'time': 1543449600, 'volume': 7084628338.02}, {'time': 1543536000, 'volume': 6718194226.05}, {'time': 1543622400, 'volume': 5512870508.45}, {'time': 1543708800, 'volume': 4973454183.26}, {'time': 1543795200, 'volume': 5225568049.72}, {'time': 1543881600, 'volume': 5001372390.05}, {'time': 1543968000, 'volume': 5057105630.83}, {'time': 1544054400, 'volume': 6122373047.4}, {'time': 1544140800, 'volume': 7305060176.54}, {'time': 1544227200, 'volume': 5387648034.48}, {'time': 1544313600, 'volume': 4699804818.45}, {'time': 1544400000, 'volume': 4563790983.49}, {'time': 1544486400, 'volume': 4534932651.77}, {'time': 1544572800, 'volume': 34182843411.18}, {'time': 1544659200, 'volume': 115879295171.43}, {'time': 1544745600, 'volume': 8253716371.57}, {'time': 1544832000, 'volume': 3313610540.37}, {'time': 1544918400, 'volume': 3870265705.93}, {'time':

In [28]:
volume_data = get_bitcoin_exchange_volume(to_ts)

{'Type': 100, 'Message': 'Got the data', 'Data': [{'time': 1543276800, 'top_tier_volume_quote': 644220149.64, 'top_tier_volume_base': 1322406686.14, 'top_tier_volume_total': 1966626835.78, 'cccagg_volume_quote': 1340143945.23, 'cccagg_volume_base': 2363545561.22, 'cccagg_volume_total': 3703689506.45, 'total_volume_quote': 2819243713.65, 'total_volume_base': 7302750887.36, 'total_volume_total': 10121994601.02}, {'time': 1543363200, 'top_tier_volume_quote': 858781337.84, 'top_tier_volume_base': 1848769973.45, 'top_tier_volume_total': 2707551311.28, 'cccagg_volume_quote': 1801233607.99, 'cccagg_volume_base': 3035414511.72, 'cccagg_volume_total': 4836648119.71, 'total_volume_quote': 3753591218.59, 'total_volume_base': 9341973466.61, 'total_volume_total': 13095564685.2}, {'time': 1543449600, 'top_tier_volume_quote': 739489218.72, 'top_tier_volume_base': 1402253202.22, 'top_tier_volume_total': 2141742420.94, 'cccagg_volume_quote': 1586274202.42, 'cccagg_volume_base': 2473407098.46, 'cccagg_v

In [31]:
# print(price_data)
print(market_volume_data)
# print(volume_data)

            time        volume
0     1543276800  6.656716e+09
1     1543363200  8.409567e+09
2     1543449600  7.084628e+09
3     1543536000  6.718194e+09
4     1543622400  5.512871e+09
...          ...           ...
1996  1715731200  3.281177e+10
1997  1715817600  2.641025e+10
1998  1715904000  2.491386e+10
1999  1715990400  1.527673e+10
2000  1716076800  1.220720e+10

[2001 rows x 2 columns]


In [34]:
save_data()


{'Response': 'Success', 'Message': '', 'HasWarning': False, 'Type': 100, 'RateLimit': {}, 'Data': {'Aggregated': False, 'TimeFrom': 1543276800, 'TimeTo': 1716076800, 'Data': [{'time': 1543276800, 'high': 3878.9, 'low': 3613.11, 'open': 3784.77, 'volumefrom': 147999.31, 'volumeto': 555212127.53, 'close': 3822.99, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1543363200, 'high': 4398.57, 'low': 3822.94, 'open': 3822.99, 'volumefrom': 175435.76, 'volumeto': 726808596.57, 'close': 4263.55, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1543449600, 'high': 4444.39, 'low': 4130.68, 'open': 4263.52, 'volumefrom': 112103.36, 'volumeto': 479575430.64, 'close': 4286.69, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1543536000, 'high': 4331.07, 'low': 3916.3, 'open': 4286.85, 'volumefrom': 121993.57, 'volumeto': 497762735.43, 'close': 4009.67, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1543622400, 'high': 4309.65, 'low': 3942.62, '